In [93]:
import os

os.chdir("C:/Users/Tanaya/Downloads/Projects/End-to-End-Machine-Learning-Project-with-MLFlow")
print("Working directory is now:", os.getcwd())


Working directory is now: C:\Users\Tanaya\Downloads\Projects\End-to-End-Machine-Learning-Project-with-MLFlow


In [94]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [95]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [96]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [98]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the directory
        Functions returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [99]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-07-11 13:17:23,477: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-11 13:17:23,493: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-11 13:17:23,501: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-11 13:17:23,501: INFO: common: created directory at artifacts]
[2025-07-11 13:17:23,509: INFO: common: created directory at artifacts/data_ingestion]
[2025-07-11 13:17:25,541: INFO: 2344350516: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d9ff3bbeb18ae65114692490179c798b2504f76c0e96bf8521dc224564f8c57d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A1C6:1C70C:AB5D1:180769:6870C18C
Accept-Ranges: bytes
Date: Fri, 11 J